# The Transformer Mistake

This is the most famous image in the world of machine learning.

![The transformer architecture](files/AIAYN.PNG "The transformer architecture")

In "Attention Is All You Need", this diagram shows the transformer architecture. However, this image is wrong. This is not  actually the transformer architecture.

Moreover, in the original paper they discuss the output of each sublayer being $LayerNorm(x+Sublayer(x))$.

Again this in incorrect. This is not the transformer which became the highly important model architecture, but rather a fatally flawed implementation.

Of course the correct sublayer output is $ x+ LayerNorm(Sublayer(x))$ where the residual connection is outside the layer norm.


## What happened?

Between the preprint of AIAYN in June 2017, and the official publication in December 2017, this flawed implementation (now known as Post-LN) was corrected in the official code (to the Pre-LN version). But, those corrections never made it into the paper. Hence the most famous paper in the field shows something that was changed before the official publication even, something that has a really significant impact in the large deep networks that would come in 2018 and 2019, before the decoder only GPT-1 was published, warping the idea of a transformer further.

You can see the correction [here](https://github.com/tensorflow/tensor2tensor/commit/d5bdfcc85fa3e10a73902974f2c0944dc51f6a33) with this particular piece of arcane code added:

```
    hparams.layer_preprocess_sequence = "n"
    hparams.layer_postprocess_sequence = "da"
```

We can follow the effects of this code into the layer_prepostprocess function [here](https://github.com/tensorflow/tensor2tensor/blob/d5bdfcc85fa3e10a73902974f2c0944dc51f6a33/tensor2tensor/layers/common_layers.py) where we see the effect of the flag 'n' is that before entering into the sublayer, we normalize, and then after exiting the layer we accumulate the residual connection with the flag 'a'. This is now exactly what we want and what would become known as the classic encoder-decoder transformer architecture.

## The transformer_n_da function

Interestingly in the code, there was a unused function called transformer_n_da which held the correct implementation (on line 632 of transformer.py). This function had been in the code for about 2 months before the main function was changed. It was added about 2 months after the preprint - see [here](https://github.com/tensorflow/tensor2tensor/commit/f5c9b17e617ea9179b7d84d36b1e8162cb369f25)

At this addition, they made the comment "Normalization on layer input instead of after the residual seems to help in learning deep networks". Remember, the test transformer they trained in AIAYN was only 6 layers (the biggest was 8) so not that deep and this failure wouldn't have been observed for the test networks they used in the paper's experiments

This comment would later be expanded in great detail in a paper in 2020 called [On Layer Normalization in the Transformer Architecture](https://arxiv.org/pdf/2002.04745.pdf). In this paper the authors establish that the Pre-LN transformers has smaller expected gradients in deep networks, showing an extra divisor on the gradient bound of $ \sqrt{N} $, where $N$ is the number of layers.
